In [1]:
import os  # Manipulate Operating System
import win32gui, win32ui, win32con, win32clipboard  # Interface with Windows
import pandas as pd  # DataFrames
import numpy as np  # Array Transformation, Math
import warnings  # Ignore Warnings
import pyautogui  # Control Mouse and Keyboard
import re  # Regular Expressions for String Parsing

# Custom WindowCapture Class
from windowcapture import WindowCapture

# Warnings to Ignore
warnings.filterwarnings("ignore", category=UserWarning)

# Set to False so mouse cursor can be controlled beyond main screen
# Used due to multiple monitor setup
pyautogui.FAILSAFE = False

# Initialize WindowCapture class with name of NetPlay window
wincap = WindowCapture(window_name="NetPlay")
screenshot = wincap.get_screenshot()


In [289]:
screenshot.shape

(582, 723, 3)

In [290]:
player_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Player.png",
    haystackImage=screenshot,
)

player_center = pyautogui.center(player_location)

mapping_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Mapping.png",
    haystackImage=screenshot,
)

mapping_center = pyautogui.center(mapping_location)

print(f"Player Center: {player_center}")
print(f"Mapping Center: {mapping_center}")

player_screen_center = wincap.get_screen_position(player_center)
mapping_screen_center = wincap.get_screen_position(mapping_center)

print(f"Player Center Relative to Window Location on Screen: {player_screen_center}")
print(f"Mapping Center Relative to Window Location on Screen: {mapping_screen_center}")


Player Center: Point(x=383, y=133)
Mapping Center: Point(x=555, y=132)
Player Center Relative to Window Location on Screen: (-1379, 414)
Mapping Center Relative to Window Location on Screen: (-1207, 413)


In [291]:
player_screen_center[0]

-1379

In [292]:
def create_player_frame(
    player_location: tuple[int, int],
    mapping_location: tuple[int, int],
    wincap: WindowCapture,
) -> pd.DataFrame:

    # Translate Player Location Tuple into X and Y variables
    player_x = player_location[0]
    player_y = player_location[1]

    # Translate Mapping Location Tuple into X and Y variables
    mapping_x = mapping_location[0]
    mapping_y = mapping_location[1]

    # Bring the window to the front with the Window Handle ID
    win32gui.SetForegroundWindow(wincap.hwnd)

    # Abstraction for pyautogui interaction
    # Helper Function to copy selected cell value and paste into variable
    def click_helper(x: int, y: int, y_move: int) -> None:
        pyautogui.click(x, y)
        pyautogui.move(0, y_move)
        pyautogui.click()
        pyautogui.hotkey("ctrl", "c")
        
    def copy_paste() -> str:
        win32clipboard.OpenClipboard()
        data = win32clipboard.GetClipboardData()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
        
        return data

    # Copy Player Data from NetPlay Window
    click_helper(x=player_x, y=player_y, y_move=25)
    player_one = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=55)
    player_two = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=85)
    player_three = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=120)
    player_four = copy_paste()

    # Copy Controller Mapping (Port) Data from NetPlay Window
    click_helper(x=mapping_x, y=mapping_y, y_move=25)
    mapping_one = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=55)
    mapping_two = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=85)
    mapping_three = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=120)
    mapping_four = copy_paste()

    # Insert Values into Pandas DataFrame
    df = pd.DataFrame(
        data={
            "Player": [player_one, player_two, player_three, player_four],
            "Port": [mapping_one, mapping_two, mapping_three, mapping_four],
        }
    )

    # Remove rows with duplicated Player values
    df = df.drop_duplicates(subset="Player", keep="first")

    # Cast string to Capitalization (string -> String | STRING -> String)
    df["Player"] = df["Player"].str.capitalize()

    df["Port"]

    return df


In [ ]:
df = create_player_frame(
    player_location=player_screen_center,
    mapping_location=mapping_screen_center,
    wincap=wincap,
)

df = df.replace("None", np.nan)

In [294]:
df

,Player,Port
0,Cooper,"GC1,2"
1,Blake,GC3
2,Connor,GC4
3,Cole,NaN


In [22]:
# Read Testing DataFrame

df = (
    pd.read_csv(r"C:\Users\Cooper\cv-mk64\img\netplay\netplay_test.csv")
    .drop(columns="Unnamed: 0")
    .dropna(axis=0, how="any")
)

df


,Player,Port
0,Cooper,"GC1,2"
1,Blake,GC3
2,Connor,GC4


In [23]:
df["Port"] = df["Port"].replace("[a-zA-Z,]", "", regex=True)


In [24]:
df

,Player,Port
0,Cooper,12
1,Blake,3
2,Connor,4


In [29]:
def validate_multiport_player(df: pd.DataFrame) -> pd.DataFrame:
    for idx, i in enumerate(df["Port"]):

        # Multiport Player has two ports
        if len(i) == 2:
            # Ports are not necessarily [1, 2] could be [2, 3] etc.
            # Index into the string below
            port_x = i[0]
            port_y = i[1]

            # Prompt user with multiple assigned ports for input
            player_x = input(f"Input Player on Port {port_x}: ")
            player_y = input(f"Input Player on Port {port_y}: ")

            # Drop original row for user with multiple ports using enum index
            odf = df.drop(idx)

            # Create temporary DataFrame with data received from input variables
            # Prompt specifies port, if data is not correct downstream data will be mismatched
            tdf = pd.DataFrame(
                data={"Player": [player_x, player_y], "Port": [port_x, port_y]}
            )

            # Combine the original and temporary DataFrame
            cdf = pd.concat(objs=[tdf, odf])

            # Cast Port values to int
            cdf["Port"] = cdf["Port"].astype(int)

            # Sort and reset the index
            cdf = cdf.sort_values(by="Port", ascending=True).reset_index(drop=True)

        elif len(i) == 3:

            # Ports are not necessarily [1, 2, 3] could be [2, 3, 4] etc.
            # Index into the string below
            port_x = i[0]
            port_y = i[1]
            port_z = i[2]

            # Prompt user with multiple assigned ports input
            player_x = input(f"Input Player on Port {port_x}: ")
            player_y = input(f"Input Player on Port {port_y}: ")
            player_z = input(f"Input Player on Port {port_z}: ")

            # Drop original row for user with multiple ports using enum index
            odf = df.drop(idx)

            # Create temporary DataFrame with data received from input variables
            # Prompt specifies port, if data is not correct downstream data will be mismatched
            tdf = pd.DataFrame(
                data={
                    "Player": [player_x, player_y, player_z],
                    "Port": [port_x, port_y, port_z],
                }
            )

            # Combine the original and temporary DataFrame
            cdf = pd.concat(objs=[tdf, odf])

            # Cast Port values to int
            cdf["Port"] = cdf["Port"].astype(int)

            # Sort and reset the index
            cdf = cdf.sort_values(by="Port", ascending=True).reset_index(drop=True)

        else:
            pass
    
    return cdf

In [30]:
player_df = validate_multiport_player(df=df)

In [31]:
player_df

,Player,Port
0,Cooper,1
1,Matt,2
2,Blake,3
3,Connor,4
